# Learning

In [42]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [43]:
MODEL = 'ResNet50'
batch_size=8
train_data_path = './CargoContainerImages/train/'
num_classes = 2
max_epoches = 10

In [44]:
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode ='categorical')

Found 127 images belonging to 2 classes.


In [45]:
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input
    model = ResNet50(weights = None, classes=num_classes)
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input
    model = VGG16(weights = None, classes = num_classes)
    output_node_name = 'predictions/Softmax'


In [46]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
16/16 [==============================] - 44s 3s/step - loss: 2.8833 - accuracy: 0.6772
Epoch 2/10
16/16 [==============================] - 43s 3s/step - loss: 1.1205 - accuracy: 0.7795
Epoch 3/10
16/16 [==============================] - 43s 3s/step - loss: 0.3510 - accuracy: 0.9213
Epoch 4/10
16/16 [==============================] - 43s 3s/step - loss: 0.3190 - accuracy: 0.8976
Epoch 5/10
16/16 [==============================] - 43s 3s/step - loss: 0.1930 - accuracy: 0.9606
Epoch 6/10
16/16 [==============================] - 45s 3s/step - loss: 0.1421 - accuracy: 0.9449
Epoch 7/10
16/16 [==============================] - 44s 3s/step - loss: 0.3784 - accuracy: 0.9370
Epoch 8/10
16/16 [==============================] - 44s 3s/step - loss: 0.4974 - accuracy: 0.9134
Epoch 9/10
16/16 [==============================] - 43s 3s/step - loss: 0.1854 - accuracy: 0.9685
Epoch 10/10
16/16 [==============================] - 43s 3s/step - loss: 0.4824 - accuracy: 0.9449


In [47]:
MODEL = 'VGG16'
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
16/16 [==============================] - 44s 3s/step - loss: 1.3500 - accuracy: 0.8189
Epoch 2/10
16/16 [==============================] - 43s 3s/step - loss: 0.6303 - accuracy: 0.8819
Epoch 3/10
16/16 [==============================] - 43s 3s/step - loss: 0.4506 - accuracy: 0.8898
Epoch 4/10
16/16 [==============================] - 43s 3s/step - loss: 0.6064 - accuracy: 0.8504
Epoch 5/10
16/16 [==============================] - 42s 3s/step - loss: 0.3943 - accuracy: 0.8819
Epoch 6/10
16/16 [==============================] - 41s 3s/step - loss: 0.0368 - accuracy: 0.9764
Epoch 7/10
16/16 [==============================] - 41s 3s/step - loss: 0.1526 - accuracy: 0.9764
Epoch 8/10
16/16 [==============================] - 41s 3s/step - loss: 0.0448 - accuracy: 0.9843
Epoch 9/10
16/16 [==============================] - 41s 3s/step - loss: 0.0033 - accuracy: 1.0000
Epoch 10/10
16/16 [==============================] - 41s 3s/step - loss: 0.1191 - accuracy: 0.9764


# Prediction

In [48]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
#import uff

In [49]:
MODEL = 'VGG16'

In [50]:
if MODEL == 'ResNet50':
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    output_node_name  = 'predictions/Softmax'
#sess = K.get_session()
#K.set_learning_phase(0)
#graph = tf.get_default_graph()
model = load_model('saved_model/' + MODEL + 'saved.h5')
model.compile(loss = 'categorical_crossenthropy', optimizer = 'adam', metrics= ['accuracy'])
#output_graph_def = tf.graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ['input_1', output_node_name])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [51]:
#uffmodel = uff.from_tensorflow(output_graph_def, [output_node_name], 
#                              output_filename = 'saved_model/' + MODEL + 'uff')

In [52]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import decode_predictions
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import decode_predictions

def predict_all(folder):
    HORIZONTAL = 0
    VERTICAL = 0
    for filename in os.listdir(folder):
        print(filename)
        fullpath = os.path.join(folder, filename)  # 폴더명과 파일명을 결합
        #print("fullpath", fullpath)  
        try:
            # 테스트할 이미지 불러오기
            img = load_img(fullpath, target_size=(224, 224)) 
            # ResNet에 입력하기 전에 이미지 전처리
            x = img_to_array(img) /255.0
            x = np.expand_dims(x, axis=0)
            #x = preprocess_input(x)

            # 이미지 분류
            preds = model.predict(x)
            print( preds)
            if preds[0][0] > preds[0][1]:
                print('HORIZONTAL')
                HORIZONTAL += 1
            else:
                print('VERTICAL')
                VERTICAL += 1
            #print('Predicted:', decode_predictions(preds, top=1)[0])
            print('HORIZONTAL: {}%, VERTICAL {}%'.format(HORIZONTAL/(HORIZONTAL+VERTICAL), VERTICAL/(HORIZONTAL+VERTICAL) ))
            print("")

        except Exception as e:
            print(e)
            print('')

In [53]:
predict_all('./CargoContainerImages/test/1/')

BHCU2088072.jpg
[[0.98542494 0.01457505]]
HORIZONTAL
HORIZONTAL: 1.0%, VERTICAL 0.0%

CISU2000076.jpg
[[2.4251055e-06 9.9999762e-01]]
VERTICAL
HORIZONTAL: 0.5%, VERTICAL 0.5%

CKLU4111544.jpg
[[0.92453974 0.07546028]]
HORIZONTAL
HORIZONTAL: 0.6666666666666666%, VERTICAL 0.3333333333333333%

CPWU700522.jpg
[[0.9876417  0.01235826]]
HORIZONTAL
HORIZONTAL: 0.75%, VERTICAL 0.25%

FCIU3558552.jpg
[[2.6638606e-06 9.9999738e-01]]
VERTICAL
HORIZONTAL: 0.6%, VERTICAL 0.4%

GVTU2086449.JPG
[[0.14847745 0.8515225 ]]
VERTICAL
HORIZONTAL: 0.5%, VERTICAL 0.5%

ICSU4925022.jpg
[[0.9487981  0.05120191]]
HORIZONTAL
HORIZONTAL: 0.5714285714285714%, VERTICAL 0.42857142857142855%

MWLU2480338.jpg
[[0.98043996 0.01956008]]
HORIZONTAL
HORIZONTAL: 0.625%, VERTICAL 0.375%

TGHU3055299.jpg
[[9.999064e-01 9.362057e-05]]
HORIZONTAL
HORIZONTAL: 0.6666666666666666%, VERTICAL 0.3333333333333333%

TRLU5438620.JPG
[[0.96497536 0.03502465]]
HORIZONTAL
HORIZONTAL: 0.7%, VERTICAL 0.3%



In [54]:
predict_all('./CargoContainerImages/test/2/')

CAIU8121016.jpg
[[1.3599703e-07 9.9999988e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

CBXU2785862.jpg
[[8.244981e-05 9.999175e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

DLSU5001789.jpg
[[5.7178804e-09 1.0000000e+00]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

FBXU0220812.jpg
[[1.5476744e-06 9.9999845e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

FXLU8425651.jpg
[[3.6686881e-06 9.9999630e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

HPCU4006609.jpg
[[9.883290e-08 9.999999e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

MTBU2033374.jpg
[[6.6958813e-07 9.9999928e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

SCTU0017000.jpg
[[5.3853514e-06 9.9999464e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

TCNU6120328.jpg
[[1.4129835e-06 9.9999857e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%

VICU5000230.jpg
[[2.4059273e-07 9.9999976e-01]]
VERTICAL
HORIZONTAL: 0.0%, VERTICAL 1.0%



In [52]:
a = 86
b=61
a/(a+b)

0.5850340136054422

# Extra

In [12]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
 
# imagenet에 미리 훈련된 ResNet50 모델 불러오기
model = ResNet50(weights='imagenet')

 
# 테스트할 이미지 불러오기
img_path = 'test.jpg'
img = load_img(img_path, target_size=(224, 224)) 
 
# ResNet에 입력하기 전에 이미지 전처리
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
 
# 이미지 분류
preds = model.predict(x)
print(preds)
print('Predicted:', decode_predictions(preds, top=3)[0])


[[3.71102676e-10 1.56775593e-09 6.33899333e-07 4.02285906e-07
  1.18995573e-07 3.43044704e-08 1.14061187e-08 1.86256255e-09
  1.97986072e-09 3.11168064e-10 1.82857895e-10 3.71855130e-10
  2.36878317e-09 3.67928488e-09 3.19695798e-10 8.83174811e-10
  3.13072803e-08 3.11225201e-09 3.42527173e-10 3.32528387e-08
  1.96200078e-09 5.57546912e-07 2.18191012e-08 1.51773691e-07
  3.04395904e-08 3.22587219e-08 9.52947854e-09 3.20642490e-09
  2.90862823e-09 5.38676925e-09 4.07433504e-10 1.42372265e-08
  8.15091727e-09 2.41507152e-08 4.86918026e-08 2.22863115e-08
  1.41772318e-08 1.19823573e-09 1.62482561e-09 4.28137570e-09
  4.24195301e-09 1.14656548e-10 2.69182348e-08 1.31458524e-08
  4.61131089e-09 1.36950917e-09 1.47408766e-10 1.41981502e-08
  8.25802760e-10 4.84810414e-09 3.96564914e-09 2.89273453e-06
  4.81442397e-09 1.40846157e-09 1.27366040e-09 4.13008294e-10
  1.33297648e-08 5.33506173e-09 2.46092835e-09 8.66849162e-11
  1.01277942e-09 4.18089989e-08 1.58589213e-08 1.74744330e-10
  6.8164